# Gradient Check

In [ ]:
# Please use python 2.7
# This notebook is based on cs231n/2016winter/assignment3
# A bit of setup

import time
import numpy as np

from gradient_check import eval_numerical_gradient, eval_numerical_gradient_array
from rnn_layers import *

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
  """ returns relative error """
  return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

# LSTM
If you read recent papers, you'll see that many people use a variant on the vanialla RNN called Long-Short Term Memory (LSTM) RNNs. Vanilla RNNs can be tough to train on long sequences due to vanishing and exploding gradiants caused by repeated matrix multiplication. LSTMs solve this problem by replacing the simple update rule of the vanilla RNN with a gating mechanism as follows.

Similar to the vanilla RNN, at each timestep we receive an input $x_t\in\mathbb{R}^D$ and the previous hidden state $h_{t-1}\in\mathbb{R}^H$; the LSTM also maintains an $H$-dimensional *cell state*, so we also receive the previous cell state $c_{t-1}\in\mathbb{R}^H$. The learnable parameters of the LSTM are an *input-to-hidden* matrix $W_x\in\mathbb{R}^{4H\times D}$, a *hidden-to-hidden* matrix $W_h\in\mathbb{R}^{4H\times H}$ and a *bias vector* $b\in\mathbb{R}^{4H}$.

At each timestep we first compute an *activation vector* $a\in\mathbb{R}^{4H}$ as $a=W_xx_t + W_hh_{t-1}+b$. We then divide this into four vectors $a_i,a_f,a_o,a_g\in\mathbb{R}^H$ where $a_i$ consists of the first $H$ elements of $a$, $a_f$ is the next $H$ elements of $a$, etc. We then compute the *input gate* $g\in\mathbb{R}^H$, *forget gate* $f\in\mathbb{R}^H$, *output gate* $o\in\mathbb{R}^H$ and *block input* $g\in\mathbb{R}^H$ as

$$
\begin{align*}
i = \sigma(a_i) \hspace{2pc}
f = \sigma(a_f) \hspace{2pc}
o = \sigma(a_o) \hspace{2pc}
g = \tanh(a_g)
\end{align*}
$$

where $\sigma$ is the sigmoid function and $\tanh$ is the hyperbolic tangent, both applied elementwise.

Finally we compute the next cell state $c_t$ and next hidden state $h_t$ as

$$
c_{t} = f\odot c_{t-1} + i\odot g \hspace{4pc}
h_t = o\odot\tanh(c_t)
$$

where $\odot$ is the elementwise product of vectors.

In the rest of the notebook we will implement the LSTM update rule and apply it to the image captioning task.

# LSTM: step forward
Implement the forward pass for a single timestep of an LSTM in the `lstm_step_forward` function in the file `rnn_layers.py`. This should be similar to the `rnn_step_forward` function, but using the LSTM update rule instead.

Once you are done, run the following to perform a simple test of your implementation. You should see errors around `1e-8` or less.

In [ ]:
N, D, H = 3, 4, 5
x = np.linspace(-0.4, 1.2, num=N*D).reshape(N, D)
prev_h = np.linspace(-0.3, 0.7, num=N*H).reshape(N, H)
prev_c = np.linspace(-0.4, 0.9, num=N*H).reshape(N, H)
Wx = np.linspace(-2.1, 1.3, num=4*D*H).reshape(D, 4 * H)
Wh = np.linspace(-0.7, 2.2, num=4*H*H).reshape(H, 4 * H)
b = np.linspace(0.3, 0.7, num=4*H)

next_h, next_c, cache = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)

expected_next_h = np.asarray([
    [ 0.24635157,  0.28610883,  0.32240467,  0.35525807,  0.38474904],
    [ 0.49223563,  0.55611431,  0.61507696,  0.66844003,  0.7159181 ],
    [ 0.56735664,  0.66310127,  0.74419266,  0.80889665,  0.858299  ]])
expected_next_c = np.asarray([
    [ 0.32986176,  0.39145139,  0.451556,    0.51014116,  0.56717407],
    [ 0.66382255,  0.76674007,  0.87195994,  0.97902709,  1.08751345],
    [ 0.74192008,  0.90592151,  1.07717006,  1.25120233,  1.42395676]])

print 'next_h error: ', rel_error(expected_next_h, next_h)
print 'next_c error: ', rel_error(expected_next_c, next_c)

# LSTM: step backward
Implement the backward pass for a single LSTM timestep in the function `lstm_step_backward` in the file `rnn_layers.py`. Once you are done, run the following to perform numeric gradient checking on your implementation. You should see errors around `1e-8` or less.

In [ ]:
N, D, H = 4, 5, 6
x = np.random.randn(N, D)
prev_h = np.random.randn(N, H)
prev_c = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

next_h, next_c, cache = lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)

dnext_h = np.random.randn(*next_h.shape)
dnext_c = np.random.randn(*next_c.shape)

fx_h = lambda x: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fh_h = lambda h: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fc_h = lambda c: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fWx_h = lambda Wx: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fWh_h = lambda Wh: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]
fb_h = lambda b: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[0]

fx_c = lambda x: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fh_c = lambda h: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fc_c = lambda c: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fWx_c = lambda Wx: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fWh_c = lambda Wh: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]
fb_c = lambda b: lstm_step_forward(x, prev_h, prev_c, Wx, Wh, b)[1]

num_grad = eval_numerical_gradient_array

dx_num = num_grad(fx_h, x, dnext_h) + num_grad(fx_c, x, dnext_c)
dh_num = num_grad(fh_h, prev_h, dnext_h) + num_grad(fh_c, prev_h, dnext_c)
dc_num = num_grad(fc_h, prev_c, dnext_h) + num_grad(fc_c, prev_c, dnext_c)
dWx_num = num_grad(fWx_h, Wx, dnext_h) + num_grad(fWx_c, Wx, dnext_c)
dWh_num = num_grad(fWh_h, Wh, dnext_h) + num_grad(fWh_c, Wh, dnext_c)
db_num = num_grad(fb_h, b, dnext_h) + num_grad(fb_c, b, dnext_c)

dx, dh, dc, dWx, dWh, db = lstm_step_backward(dnext_h, dnext_c, cache)

print 'dx error: ', rel_error(dx_num, dx)
print 'dh error: ', rel_error(dh_num, dh)
print 'dc error: ', rel_error(dc_num, dc)
print 'dWx error: ', rel_error(dWx_num, dWx)
print 'dWh error: ', rel_error(dWh_num, dWh)
print 'db error: ', rel_error(db_num, db)

# LSTM: forward
In the function `lstm_forward` in the file `rnn_layers.py`, implement the `lstm_forward` function to run an LSTM forward on an entire timeseries of data.

When you are done run the following to check your implementation. You should see an error around `1e-7`.

In [ ]:
N, D, H, T = 2, 5, 4, 3
x = np.linspace(-0.4, 0.6, num=N*T*D).reshape(N, T, D)
h0 = np.linspace(-0.4, 0.8, num=N*H).reshape(N, H)
Wx = np.linspace(-0.2, 0.9, num=4*D*H).reshape(D, 4 * H)
Wh = np.linspace(-0.3, 0.6, num=4*H*H).reshape(H, 4 * H)
b = np.linspace(0.2, 0.7, num=4*H)

h, cache = lstm_forward(x, h0, Wx, Wh, b)

expected_h = np.asarray([
 [[ 0.01764008,  0.01823233,  0.01882671,  0.0194232 ],
  [ 0.11287491,  0.12146228,  0.13018446,  0.13902939],
  [ 0.31358768,  0.33338627,  0.35304453,  0.37250975]],
 [[ 0.45767879,  0.4761092,   0.4936887,   0.51041945],
  [ 0.6704845,   0.69350089,  0.71486014,  0.7346449 ],
  [ 0.81733511,  0.83677871,  0.85403753,  0.86935314]]])

print 'h error: ', rel_error(expected_h, h)

# LSTM: backward
Implement the backward pass for an LSTM over an entire timeseries of data in the function `lstm_backward` in the file `rnn_layers.py`. When you are done run the following to perform numeric gradient checking on your implementation. You should see errors around `1e-8` or less.

In [ ]:
from rnn_layers import lstm_forward, lstm_backward

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

out, cache = lstm_forward(x, h0, Wx, Wh, b)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, db = lstm_backward(dout, cache)

fx = lambda x: lstm_forward(x, h0, Wx, Wh, b)[0]
fh0 = lambda h0: lstm_forward(x, h0, Wx, Wh, b)[0]
fWx = lambda Wx: lstm_forward(x, h0, Wx, Wh, b)[0]
fWh = lambda Wh: lstm_forward(x, h0, Wx, Wh, b)[0]
fb = lambda b: lstm_forward(x, h0, Wx, Wh, b)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)

print 'dx error: ', rel_error(dx_num, dx)
print 'dh0 error: ', rel_error(dx_num, dx)
print 'dWx error: ', rel_error(dx_num, dx)
print 'dWh error: ', rel_error(dx_num, dx)
print 'db error: ', rel_error(dx_num, dx)

# SRU: step backward

In [ ]:
from sru import *

N, D, H = 4, 5, 6
x = np.random.randn(N, D)
prev_c = np.random.randn(N, H)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)

next_h, next_c, cache = sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)

dnext_h = np.random.randn(*next_h.shape)
dnext_c = np.random.randn(*next_c.shape)


fx = lambda x: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fc = lambda prev_c: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fW = lambda W: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fWf = lambda Wf: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fWr = lambda Wr: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fWh = lambda Wh: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fbf = lambda bf: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]
fbr = lambda br: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[0]

fx_c = lambda x: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fc_c = lambda prev_c: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fW_c = lambda W: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fWf_c = lambda Wf: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fWr_c = lambda Wr: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fWh_c = lambda Wh: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fbf_c = lambda bf: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]
fbr_c = lambda br: sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)[1]

num_grad = eval_numerical_gradient_array

dx_num = num_grad(fx, x, dnext_h) + num_grad(fx_c, x, dnext_c)
dc_num = num_grad(fc, prev_c, dnext_h) + num_grad(fc_c, prev_c, dnext_c)
dW_num = num_grad(fW, W, dnext_h) + num_grad(fW_c, W, dnext_c)
dWf_num = num_grad(fWf, Wf, dnext_h) + num_grad(fWf_c, Wf, dnext_c)
dWr_num = num_grad(fWr, Wr, dnext_h) + num_grad(fWr_c, Wr, dnext_c)
dWh_num = num_grad(fWh, Wh, dnext_h) + num_grad(fWh_c, Wh, dnext_c)
dbf_num = num_grad(fbf, bf, dnext_h) + num_grad(fbf_c, bf, dnext_c)
dbr_num = num_grad(fbr, br, dnext_h) + num_grad(fbr_c, br, dnext_c)

dx, dpc, dW, dWf, dWr, dWh, dbf, dbr = sru_step_backward(dnext_h, dnext_c, cache)
print 'dx error: ', rel_error(dx_num, dx)
print 'dc error: ', rel_error(dc_num, dpc)
print 'dW error: ', rel_error(dW_num, dW)
print 'dWf error: ', rel_error(dWf_num, dWf)
print 'dWr error: ', rel_error(dWr_num, dWr)
print 'dWh error: ', rel_error(dWh_num, dWh)
print 'dbf error: ', rel_error(dbf_num, dbf)
print 'dbr error: ', rel_error(dbr_num, dbr)

# SRU: backward

In [ ]:
from sru import *

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)

out, cache = sru_forward(x, W, Wf, Wr, Wh, bf, br)

dout = np.random.randn(*out.shape)

dx, dW, dWf, dWr, dWh, dbf, dbr = sru_backward(dout, cache)

fx = lambda x: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]
fW = lambda W: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]
fWf = lambda Wf: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]
fWr = lambda Wr: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]
fWh = lambda Wh: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]
fbf = lambda bf: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]
fbr = lambda br: sru_forward(x, W, Wf, Wr, Wh, bf, br)[0]

num_grad = eval_numerical_gradient_array

dx_num = num_grad(fx, x, dout)
dW_num = num_grad(fW, W, dout)
dWf_num = num_grad(fWf, Wf, dout)
dWr_num = num_grad(fWr, Wr, dout)
dWh_num = num_grad(fWh, Wh, dout)
dbf_num = num_grad(fbf, bf, dout)
dbr_num = num_grad(fbr, br, dout)

print 'dx error: ', rel_error(dx_num, dx)
print 'dc error: ', rel_error(dc_num, dpc)
print 'dW error: ', rel_error(dW_num, dW)
print 'dWf error: ', rel_error(dWf_num, dWf)
print 'dWr error: ', rel_error(dWr_num, dWr)
print 'dWh error: ', rel_error(dWh_num, dWh)
print 'dbf error: ', rel_error(dbf_num, dbf)
print 'dbr error: ', rel_error(dbr_num, dbr)

# SRU vs LSTM

In [ ]:
from sru import *
from time import time

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)

MAX = 10000
start = time()
for i in range(MAX):
    out, cache = sru_forward(x, W, Wf, Wr, Wh, bf, br)
end = time()
print 'SRU: ', (end-start)

from rnn_layers import lstm_forward, lstm_backward

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, H)
Wx = np.random.randn(D, 4 * H)
Wh = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)

start = time()
for i in range(MAX):
    out, cache = lstm_forward(x, h0, Wx, Wh, b)
end = time()
print 'LSTM: ', (end-start)

# SRU step: compact weight matrix

In [ ]:
from sru import *

N, D, H = 4, 5, 6
x = np.random.randn(N, D)
prev_c = np.random.randn(N, H)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)

W_ALL = np.hstack((W, Wf, Wr, Wh))

next_h, next_c, cache = sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)
next_h_fast, next_c_fast, cache_fast = sru_step_forward_fast(x, prev_c, W_ALL, bf, br)
print 'next_h fast relative error:', rel_error(next_h, next_h_fast)
print 'next_c fast relative error:', rel_error(next_c, next_c_fast)

MAX = 100000
start = time()
for i in range(MAX):
    next_h, next_c, cache = sru_step_forward(x, prev_c, W, Wf, Wr, Wh, bf, br)
end = time()
print 'time 1:', end-start

start = time()
for i in range(MAX):
    next_h_fast, next_c_fast, cache_fast = sru_step_forward_fast(x, prev_c, W_ALL, bf, br)
end = time()
print 'time 2:', end-start

dnext_h = np.random.randn(*next_h.shape)
dnext_c = np.random.randn(*next_c.shape)
dx, dpc, dW, dWf, dWr, dWh, dbf, dbr = sru_step_backward(dnext_h, dnext_c, cache)
dxfast, dpcfast, dWfast, dbffast, dbrfast = sru_step_backward_fast(dnext_h, dnext_c, cache_fast)
print rel_error(dx, dxfast)
print rel_error(dpc, dpcfast)
print rel_error(dWfast, np.hstack((dW, dWf, dWr, dWh)))
print rel_error(dbf, dbffast)
print rel_error(dbr, dbrfast)

# SRU compact forward

In [ ]:
from sru import *
from time import time

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)


MAX = 1000
start = time()
for i in range(MAX):
    out, cache = sru_forward(x, W, Wf, Wr, Wh, bf, br)
end = time()
print 'SRU: ', (end-start)


W_ALL = np.hstack((W, Wf, Wr, Wh))
start = time()
for i in range(MAX):
    out_fast, cache_fast = sru_forward_fast(x, W_ALL, bf, br)
end = time()
print 'SRU compact: ', (end-start)

print (out - out_fast) < 1e-10
print rel_error(out, out_fast)

# SRU compact backward

In [ ]:
from sru import *

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)

W_ALL = np.hstack((W, Wf, Wr, Wh))

out, cache = sru_forward(x, W, Wf, Wr, Wh, bf, br)
out_fast, cache_fast = sru_forward_fast(x, W_ALL, bf, br)
print rel_error(out, out_fast)

dout = np.random.randn(*out.shape)

dx, dW, dWf, dWr, dWh, dbf, dbr = sru_backward(dout, cache)
dxfast, dWfast, dbffast, dbrfast = sru_backward_fast(dout, cache_fast)

print rel_error(dx, dxfast)
print rel_error(dWfast, np.hstack((dW, dWf, dWr, dWh)))
print rel_error(dbf, dbffast)
print rel_error(dbr, dbrfast)

# SRU compact backward advance

In [ ]:
from sru import *

N, D, T, H = 2, 3, 10, 6

x = np.random.randn(N, T, D)
W = np.random.randn(D, H)
Wf = np.random.randn(D, H)
Wr = np.random.randn(D, H)
Wh = np.random.randn(D, H)
bf = np.random.randn(H)
br = np.random.randn(H)

W_ALL = np.hstack((W, Wf, Wr, Wh))

out, cache = sru_forward(x, W, Wf, Wr, Wh, bf, br)
out_fast, cache_fast = sru_forward_fast(x, W_ALL, bf, br)
print rel_error(out, out_fast)

dout = np.random.randn(*out.shape)

dx, dW, dWf, dWr, dWh, dbf, dbr = sru_backward(dout, cache)
dxfast, dWfast, dbffast, dbrfast = sru_backward_fast(dout, cache_fast)
dxfasta, dWfasta, dbffasta, dbrfasta = sru_backward_fast_advance(dout, cache_fast)

print rel_error(dx, dxfast)
print rel_error(dWfast, np.hstack((dW, dWf, dWr, dWh)))
print rel_error(dbf, dbffast)
print rel_error(dbr, dbrfast)

print rel_error(dx, dxfasta)
print rel_error(dWfasta, np.hstack((dW, dWf, dWr, dWh)))
print rel_error(dbf, dbffasta)
print rel_error(dbr, dbrfasta)

# numpy SRU and Kaldi SRU debug

In [ ]:
np.set_printoptions(suppress=True)
np.set_printoptions(precision=8)

N, D, H, T = 2, 3, 3, 3

x = np.linspace(-0.4, 0.6, num=N*T*D).reshape(N, T, D)
W = np.linspace(-0.2, 0.9, num=4*D*H).reshape(D, 4 * H)
bf = np.linspace(0.2, 0.7, num=H)
br = np.linspace(0.2, 0.7, num=H)

print 'x', np.reshape(np.transpose(x, (1, 0, 2)), (-1, x.shape[-1]))

out_fast, cache_fast = sru_forward_fast(x, W, bf, br)
print 'x', x
print 'W.T', W.T
print 'bf', bf
print 'br', br
print 'out', out_fast
print 'transpose out\n',  np.reshape(np.transpose(out_fast, (1, 0, 2)), (-1, out_fast.shape[-1]))


dout = np.linspace(-0.6, 0.6, num=N*T*H).reshape(N, T, H)
dxfasta, dWfasta, dbffasta, dbrfasta = sru_backward_fast_advance(dout, cache_fast)
print 'dout transpose\n', np.reshape(np.transpose(dout, (1, 0, 2)), (-1, dout.shape[-1]))
print 'dxfasta transpose\n', np.reshape(np.transpose(dxfasta, (1, 0, 2)), (-1, dxfasta.shape[-1]))

# TF-LSTM forward step and backward step

In [ ]:
from tflstm import *

N, D, H = 4, 5, 6
F, S = 2, 1
B = (D-F)/S+1

x = np.random.randn(N, D)
prev_h = np.random.randn(N, B*H)
prev_c = np.random.randn(N, B*H)
Wx = np.random.randn(F, 4 * H)
Wh = np.random.randn(H, 4 * H)
Wk = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)
pi = np.random.randn(H)
pf = np.random.randn(H)
po = np.random.randn(H)

next_h, next_c, cache = tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)

dnext_h = np.random.randn(*next_h.shape)
dnext_c = np.random.randn(*next_c.shape)

fx_h = lambda x: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fh_h = lambda h: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fc_h = lambda c: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWx_h = lambda Wx: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWh_h = lambda Wh: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWk_h = lambda Wk: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fb_h = lambda b: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpi_h = lambda pi: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpf_h = lambda pf: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpo_h = lambda po: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]

fx_c = lambda x: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fh_c = lambda h: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fc_c = lambda c: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fWx_c = lambda Wx: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fWh_c = lambda Wh: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fWk_c = lambda Wk: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fb_c = lambda b: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fpi_c = lambda pi: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fpf_c = lambda pf: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]
fpo_c = lambda po: tflstm_forward_origin_step_pp(x, prev_h, prev_c, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[1]

num_grad = eval_numerical_gradient_array

dx_num = num_grad(fx_h, x, dnext_h) + num_grad(fx_c, x, dnext_c)
dh_num = num_grad(fh_h, prev_h, dnext_h) + num_grad(fh_c, prev_h, dnext_c)
dc_num = num_grad(fc_h, prev_c, dnext_h) + num_grad(fc_c, prev_c, dnext_c)
dWx_num = num_grad(fWx_h, Wx, dnext_h) + num_grad(fWx_c, Wx, dnext_c)
dWh_num = num_grad(fWh_h, Wh, dnext_h) + num_grad(fWh_c, Wh, dnext_c)
dWk_num = num_grad(fWk_h, Wk, dnext_h) + num_grad(fWk_c, Wk, dnext_c)
db_num = num_grad(fb_h, b, dnext_h) + num_grad(fb_c, b, dnext_c)
dpi_num = num_grad(fpi_h, pi, dnext_h) + num_grad(fpi_c, pi, dnext_c)
dpf_num = num_grad(fpf_h, pf, dnext_h) + num_grad(fpf_c, pf, dnext_c)
dpo_num = num_grad(fpo_h, po, dnext_h) + num_grad(fpo_c, po, dnext_c)

dx, dh, dc, dWx, dWh, dWk, db, dpi, dpf, dpo = tflstm_backward_advance_origin_step_pp(dnext_h, dnext_c, cache)

print 'dx error: ', rel_error(dx_num, dx)
print 'dh error: ', rel_error(dh_num, dh)
print 'dc error: ', rel_error(dc_num, dc)
print 'dWx error: ', rel_error(dWx_num, dWx)
print 'dWh error: ', rel_error(dWh_num, dWh)
print 'dWk error: ', rel_error(dWk_num, dWk)
print 'db error: ', rel_error(db_num, db)
print 'dpi error: ', rel_error(dpi_num, dpi)
print 'dpf error: ', rel_error(dpf_num, dpf)
print 'dpo error: ', rel_error(dpo_num, dpo)

# TF-LSTM forward (use step impl) and backward (not advance time)

In [ ]:
from tflstm import *

N, D, T, H = 2, 3, 10, 6
F, S = 2, 1
B = (D-F)/S+1

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, B*H)
Wx = np.random.randn(F, 4 * H)
Wh = np.random.randn(H, 4 * H)
Wk = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)
pi = np.random.randn(H)
pf = np.random.randn(H)
po = np.random.randn(H)

out, cache = tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, dWk, db, dpi, dpf, dpo,  = tflstm_backward_pp(dout, cache)

fx = lambda x: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fh0 = lambda h0: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWx = lambda Wx: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWh = lambda Wh: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWk = lambda Wk: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fb = lambda b: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpi = lambda pi: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpf = lambda pf: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpo = lambda po: tflstm_forward_origin2_pp(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
dWk_num = eval_numerical_gradient_array(fWk, Wk, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)
dpi_num = eval_numerical_gradient_array(fpi, pi, dout)
dpf_num = eval_numerical_gradient_array(fpf, pf, dout)
dpo_num = eval_numerical_gradient_array(fpo, po, dout)


print 'dx error: ', rel_error(dx_num, dx)
print 'dh0 error: ', rel_error(dh0_num, dh0)
print 'dWx error: ', rel_error(dWx_num, dWx)
print 'dWh error: ', rel_error(dWh_num, dWh)
print 'dWk error: ', rel_error(dWk_num, dWk)
print 'db error: ', rel_error(db_num, db)
print 'dpi error: ', rel_error(dpi_num, dpi)
print 'dpf error: ', rel_error(dpf_num, dpf)
print 'dpo error: ', rel_error(dpo_num, dpo)

# TF-LSTM forward unfold and backward advance time

In [ ]:
from tflstm import *

N, D, T, H = 2, 3, 10, 6
F, S = 2, 1
B = (D-F)/S+1

x = np.random.randn(N, T, D)
h0 = np.random.randn(N, B*H)
Wx = np.random.randn(F, 4 * H)
Wh = np.random.randn(H, 4 * H)
Wk = np.random.randn(H, 4 * H)
b = np.random.randn(4 * H)
pi = np.random.randn(H)
pf = np.random.randn(H)
po = np.random.randn(H)

out, cache = tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)

dout = np.random.randn(*out.shape)

dx, dh0, dWx, dWh, dWk, db, dpi, dpf, dpo,  = tflstm_backward_pp_advance(dout, cache)

fx = lambda x: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fh0 = lambda h0: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWx = lambda Wx: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWh = lambda Wh: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fWk = lambda Wk: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fb = lambda b: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpi = lambda pi: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpf = lambda pf: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]
fpo = lambda po: tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)[0]

dx_num = eval_numerical_gradient_array(fx, x, dout)
dh0_num = eval_numerical_gradient_array(fh0, h0, dout)
dWx_num = eval_numerical_gradient_array(fWx, Wx, dout)
dWh_num = eval_numerical_gradient_array(fWh, Wh, dout)
dWk_num = eval_numerical_gradient_array(fWk, Wk, dout)
db_num = eval_numerical_gradient_array(fb, b, dout)
dpi_num = eval_numerical_gradient_array(fpi, pi, dout)
dpf_num = eval_numerical_gradient_array(fpf, pf, dout)
dpo_num = eval_numerical_gradient_array(fpo, po, dout)


print 'dx error: ', rel_error(dx_num, dx)
print 'dh0 error: ', rel_error(dh0_num, dh0)
print 'dWx error: ', rel_error(dWx_num, dWx)
print 'dWh error: ', rel_error(dWh_num, dWh)
print 'dWk error: ', rel_error(dWk_num, dWk)
print 'db error: ', rel_error(db_num, db)
print 'dpi error: ', rel_error(dpi_num, dpi)
print 'dpf error: ', rel_error(dpf_num, dpf)
print 'dpo error: ', rel_error(dpo_num, dpo)

# numpy TF-LSTM and Kaldi TF-LSTM debug

In [ ]:
from tflstm import *
np.set_printoptions(suppress=True)
np.set_printoptions(precision=8)

N, D, T, H = 2, 3, 3, 3
F, S = 2, 1
B = (D-F)/S+1

x = np.linspace(-0.4, 0.6, num=N*T*D).reshape(N, T, D)
h0 = np.zeros((N, B*H))
Wx = np.linspace(-0.2, 0.9, num=F*4*H).reshape(F, 4 * H)
Wh = np.linspace(-0.3, 0.8, num=H*4*H).reshape(H, 4 * H)
Wk = np.linspace(-0.4, 0.7, num=H*4*H).reshape(H, 4 * H)
b = np.linspace(0.2, 0.7, num=4*H)
pi = np.linspace(0.1, 0.6, num=H)
pf = np.linspace(0.2, 0.7, num=H)
po = np.linspace(0.3, 0.8, num=H)

print 'x\n', x
print 'transpose x\n', np.reshape(np.transpose(x, (1, 0, 2)), (-1, x.shape[-1]))
print 'h0\n', h0
print 'Wx.T\n', Wx.T
print 'Wh.T\n', Wh.T
print 'Wk.T\n', Wk.T
print 'b\n', b
print 'pi\n', pi
print 'pf\n', pf
print 'po\n', po

out, cache = tflstm_forward_origin2_pp_unfold(x, h0, Wx, Wh, Wk, b, pi, pf, po, F, S, H)

print 'out\n', out
print 'transpose out\n', np.reshape(np.transpose(out, (1, 0, 2)), (-1, out.shape[-1]))

dout = np.linspace(-0.6, 0.6, num=N*T*B*H).reshape(N, T, B*H)
dx, dh0, dWx, dWh, dWk, db, dpi, dpf, dpo,  = tflstm_backward_pp_advance(dout, cache)

print 'dout transpose\n', np.reshape(np.transpose(dout, (1, 0, 2)), (-1, dout.shape[-1]))
print 'dx transpose\n', np.reshape(np.transpose(dx, (1, 0, 2)), (-1, dx.shape[-1]))
print 'dWx.T', dWx.T
print 'dWh.T', dWh.T
print 'dWk.T', dWk.T
print 'db', db
print 'dpi', dpi
print 'dpf', dpf
print 'dpo', dpo